# Keras Model

Before you begin using Keras, import the required modules. If you are having problems with this step, make sure you have installed all dependencies found in requirements.txt and have selected the correct Python kernel (in the top menu, "Kernel"->"Change kernel"). If you are using virtualenv, your kernel will not show up unless you follow the instructions found [here](https://stackoverflow.com/questions/37891550/jupyter-notebook-running-kernel-in-different-env).

In [3]:
import numpy as np
import pandas as pd

import keras
from keras.models import Sequential, Model
from keras.layers import Input, Dense
from keras.optimizers import SGD

from sklearn.preprocessing import StandardScaler

## Data Preprocessing

First, we want to grab the dataset from the CSV file. Load it as a Pandas Dataframe so we can easily work with it in further steps.

In [4]:
def get_dataset(path):
    dataset = pd.read_csv(path)
    np.random.seed(42)
    dataset = dataset.reindex(np.random.permutation(dataset.index))
    return dataset

dataset = get_dataset('data/dataset.csv')
dataset.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
204,87930,B,12.47,18.60,81.09,481.9,0.09965,0.1058,0.08005,0.03821,...,24.64,96.05,677.9,0.1426,0.2378,0.2671,0.10150,0.3014,0.08750,NaN
70,859575,M,18.94,21.31,123.60,1130.0,0.09009,0.1029,0.10800,0.07951,...,26.58,165.90,1866.0,0.1193,0.2336,0.2687,0.17890,0.2551,0.06589,NaN
131,8670,M,15.46,19.48,101.70,748.9,0.10920,0.1223,0.14660,0.08087,...,26.00,124.90,1156.0,0.1546,0.2394,0.3791,0.15140,0.2837,0.08019,NaN
431,907915,B,12.40,17.68,81.47,467.8,0.10540,0.1316,0.07741,0.02799,...,22.91,89.61,515.8,0.1450,0.2629,0.2403,0.07370,0.2556,0.09359,NaN
540,921385,B,11.54,14.44,74.65,402.9,0.09984,0.1120,0.06737,0.02594,...,19.68,78.78,457.8,0.1345,0.2118,0.1797,0.06918,0.2329,0.08134,NaN


Now, we'll write functions to get the data and the labels (i.e. the x and the y). We want to make sure these return NumPy arrays so they can be passed into Keras models.

In [8]:
def get_data(dataset):
    data = np.array(dataset.as_matrix()[:, 2:-1], dtype=np.float64)
    return data

def get_labels(dataset):
    diagnoses = dataset['diagnosis'].map({'M':1, 'B':0})
    return np.array(diagnoses.as_matrix(), dtype=np.uint8)

data, labels = get_data(dataset), get_labels(dataset)
print(data.shape, labels.shape)

(569, 30) (569,)


Finally, we'll split the data and labels into train, validation, and test sets.

In [9]:
def split_data(data, labels, split):
    train_ratio, val_ratio, test_ratio = split
    num_examples = labels.shape[0]
    train_bound, val_bound = int(train_ratio*num_examples), int(train_ratio*num_examples) + int(val_ratio*num_examples)
    
    train = {'data': data[:train_bound], 'labels': labels[:train_bound]}
    val = {'data': data[train_bound:val_bound], 'labels': labels[train_bound:val_bound]}
    test = {'data': data[val_bound:], 'labels': labels[val_bound:]}
    
    return train, val, test
    
train, val, test = split_data(data, labels, (.7, .2, .1))

Next, we want to scale our data so that each feature has mean 0 and variance 1. This is useful because it improves the stability of training our neural network. 

In [10]:
scaler = StandardScaler()
scaler.fit(train['data'])
train['data'] = scaler.transform(train['data'])
val['data'] = scaler.transform(val['data'])
test['data'] = scaler.transform(test['data'])

## Model Architecture and Training

Let's define our model's architecture first. We want a 4-layer Fully-Connected Network that can be used for binary classification.

In [11]:
print(train['data'][:10], train["labels"][:10])

from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

# baseline_model = SVC(kernel='poly', degree=2)
# baseline_model.fit(train['data'], train['labels'])
# print(baseline_model.score(val['data'], val['labels']))

baseline_mlp = MLPClassifier()
baseline_mlp.fit(train['data'], train['labels'])
print(baseline_mlp.score(train['data'], train['labels']))
print(baseline_mlp.predict(train['data']))
print(baseline_mlp.score(val['data'], val['labels']))
print(baseline_mlp.predict(val['data']))

[[ -4.51093897e-01  -1.74980482e-01  -4.27986424e-01  -4.82623057e-01
    2.71028366e-01   4.02532786e-02  -9.62593209e-02  -2.53758788e-01
    3.92284204e-01   1.11635089e-01  -1.87708345e-02  -3.43117512e-01
   -1.88953747e-01  -2.47729895e-01  -1.67306731e-02  -3.61543866e-01
   -1.52759925e-01  -1.97128787e-01  -3.43368170e-01  -8.07385172e-02
   -2.53564185e-01  -1.81859419e-01  -3.17894877e-01  -3.48243160e-01
    4.86947699e-01  -9.81702448e-02  -9.74849555e-03  -1.67030262e-01
    1.68017627e-01   1.95903816e-01]
 [  1.37951098e+00   4.59877055e-01   1.31564541e+00   1.38662702e+00
   -4.15552286e-01  -1.37439950e-02   2.48163253e-01   7.91728780e-01
   -8.07874597e-01  -1.15369273e+00   1.58622948e+00  -7.73899212e-01
    1.53011497e+00   1.57208349e+00  -8.83138857e-01  -5.09192535e-01
   -2.85011489e-01   3.61097340e-01  -8.13550564e-01  -7.73305407e-01
    1.76755742e+00   1.39101734e-01   1.72973556e+00   1.73861640e+00
   -5.47610608e-01  -1.25117612e-01  -2.13205357e-03 

/Users/Karan/tf3.5/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [333]:
def create_model():
    
    model = Sequential()
    model.add(Dense(16, activation='relu', input_dim=30))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(4, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    sgd = SGD(lr=0.001)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

model = create_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_109 (Dense)            (None, 16)                496       
_________________________________________________________________
dense_110 (Dense)            (None, 8)                 136       
_________________________________________________________________
dense_111 (Dense)            (None, 4)                 36        
_________________________________________________________________
dense_112 (Dense)            (None, 1)                 5         
Total params: 673
Trainable params: 673
Non-trainable params: 0
_________________________________________________________________


Now let's train our model! We want to not only train the model with many epochs, but also print the validation set accuracy at every epoch.

In [335]:
def train_model(model, train, val, num_epochs):
    model.fit(train['data'], train['labels'], epochs=num_epochs, verbose=1, batch_size=16)
    results = model.predict(val['data'])
    print([0 if x < .5 else 1 for x in results])
    score = model.evaluate(val['data'], val['labels'], batch_size=16, verbose=0)
    print("Validation accuracy: %.2f\n" % (score[1]))
        
train_model(model, train, val, 25)

Epoch 1/25
512/512 [==============================] - 0s - loss: 7.4927e-04 - acc: 1.0000     
Epoch 2/25
512/512 [==============================] - 0s - loss: 7.0824e-04 - acc: 1.0000     
Epoch 3/25
512/512 [==============================] - 0s - loss: 6.8463e-04 - acc: 1.0000     
Epoch 4/25
512/512 [==============================] - 0s - loss: 6.5316e-04 - acc: 1.0000     
Epoch 5/25
512/512 [==============================] - 0s - loss: 6.3599e-04 - acc: 1.0000     
Epoch 6/25
512/512 [==============================] - 0s - loss: 6.4775e-04 - acc: 1.0000     
Epoch 7/25
512/512 [==============================] - 0s - loss: 5.8751e-04 - acc: 1.0000     
Epoch 8/25
512/512 [==============================] - 0s - loss: 6.1608e-04 - acc: 1.0000     
Epoch 9/25
512/512 [==============================] - 0s - loss: 5.6789e-04 - acc: 1.0000     
Epoch 10/25
512/512 [==============================] - 0s - loss: 5.5468e-04 - acc: 1.0000     
Epoch 11/25
512/512 [============================

Try playing around with hyperparameters like the learning rate, size of the hidden layers, number of epochs, etc. until you get a model that you are satisfied with! Use validation accuracy to compare performance across different model configurations. Once you're done configuring, try testing on a completely unseen dataset to get a good idea of how your model will perform for unseen data:

In [60]:
test_score = model.evaluate(test['data'], test['labels'], batch_size=128)
print("Test accuracy: %.2f" % (test_score[1]))

58/58 [==============================] - 0s
Test accuracy: 0.36
